** Compression **

In [ ]:
def compress_file(filename):
    with open (filename, 'r', encoding='utf-8') as f:
        with open (filename+'-short', 'w', encoding='utf-8') as outp:
            for side, word1, word2 in nodes_from_file(filename):
                string = str(side) + '\t' + compress(word1).write(mode='bi') + '\t' + compress(word2).write(mode='bi') + '\n'
                outp.write(string)        

In [ ]:
%time compress_file('spa-eng')

In [ ]:
def compress(word):
    s = list(sorted(word.s, key=len))
    length = len(s[0]) - 1
    if length == -1: print('Alarm!\t', word)
    s2 = []
    for key, value in enumerate(s[-1]):
        if key > length: s2.append('('+value+')')
        else: s2.append(value)
    word.s = [Tags(s2)]
    return word

In [ ]:
word = Word('capital','spa',[Tags(['n','f']), Tags('n'), Tags(['n','f','sg'])])

In [ ]:
compress(word)

In [ ]:
def get_relevant_languages(source, target):
    G = nx.Graph()
    with open ('./files/stats.csv', 'r', encoding='utf-8') as f:
        for line in f:
            data = line.split('\t')
            coef = 1/math.log10(10+float(data[2])*float(data[3])*float(data[4]))
            if coef < 1:
                G.add_edge(data[0], data[1], weight=coef)
    if (source, target) in G.edges():
        #print ('There is {}-{} dictionary'.format(source, target))
        G.remove_edge(source, target)
    medium = set(G.nodes()) - set([source,target])
    s_dist, s_path = nx.single_source_dijkstra(G, source)
    t_dist, t_path = nx.single_source_dijkstra(G, target)
    result = {}
    for node in G.nodes():
        if node in s_dist and node in t_dist:
            result[node] = (s_dist[node]+t_dist[node], s_path[node]+list(reversed(t_path[node]))[1:])
    with open ('language_list.csv','w', encoding='utf-8') as f:
        for i in sorted(result, key=result.get):
            f.write(str(result[i][0])+'\t'+str(i)+'\t:\t'+' '.join(result[i][1])+'\n')
